# Dataset creation for multiple dimension from Raw File

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sha2, concat_ws
from pyspark.sql.functions import *
spark  = SparkSession.builder\
                    .appName("lendingClubProject")\
                    .config("spark.sql.warehouse.dir","/user/anil/warehouse")\
                    .enableHiveSupport()\
                    .master('yarn')\
                    .getOrCreate()

In [2]:
rawData = spark.read.format('csv')\
                        .option("header","true")\
                        .option("inferSchema","true")\
                        .load('/public/anil/datasets/accepted_2007_to_2018Q4.csv')

In [3]:
# member_id  is null so creating a new column using sha2 which is used to compute the hash values for each customer using existing columns of a dataset 
loansRawData = rawData.withColumn("emp_id", sha2(concat_ws("||", *["emp_title", "emp_length", "home_ownership", "annual_inc", "zip_code", "addr_state", "grade", "sub_grade","verification_status"]), 256))

In [4]:
loansRawData.count()

2260701

In [5]:
loansRawData.select('emp_id').distinct().count()

2257384

In [7]:
df = loansRawData.groupBy('emp_id').count()
df.filter('count > 1').orderBy(desc('count')).show(truncate = False)

+----------------------------------------------------------------+-----+
|emp_id                                                          |count|
+----------------------------------------------------------------+-----+
|e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855|33   |
|e4c167053d54182305bf5ea081111e06bf298584cfcaebe792744eee19461f7f|5    |
|76b577467eda5bdbc1f3640c35e2f84284bebc20eb1002a565776fd9fbb9b4ef|4    |
|3f87585a20f702838d6869aa59e7016e48592c80cf433dd53ea22648f482cc5c|4    |
|ad8e5d384dae17e06ed14adcc84d9a8076cad1e618f3843d42c24f45f41264d8|4    |
|3ae415acd6bbfaac1d77f77580ebdf95ebaa02026e56ab72d6eebcfbaebb2d76|3    |
|f54295a60946dedad3bd684accb36dbf1d91a9304623868e05ec78473e35c496|3    |
|f9fc581aae696e0bdf19d3b63c062e1b8a9d419015f6fdc82c786667b8f5dd82|3    |
|c92062bb371842b3dcff4054af3af22197fdea3ac94c97b53be7fb5d206ce750|3    |
|22593a1870543b2dbce8f4ea5effcb7959c2f2069cfbfea5aef35af3256534db|3    |
|5d52e7773cb0efff384ba77d5fda294c5157826c5f70bb926f

In [27]:
display(loansRawData.filter("emp_id == 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'"))

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,emp_id
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c44298fc1c149...
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c442

In [ ]:
#duplicate values count for each member

In [20]:
#creating dataset for customers 
customersData = loansRawData.withColumn('country',lit('USA'))\
                            .withColumnRenamed('emp_id','member').select('member','emp_title','emp_length','home_ownership','annual_inc','addr_state','zip_code','country','grade','sub_grade','verification_status','tot_hi_cred_lim','application_type','annual_inc_joint','verification_status_joint')
customersData.repartition(1).write.format('csv')\
                            .option('header','true')\
                            .mode('overwrite')\
                            .option('path','/user/anil/lendingClubProject/customers')\
                            .save()

In [19]:
#creating dataset for loans 
loans = loansRawData.withColumnRenamed('id','loan_id')\
                            .withColumnRenamed('emp_id','member').select('loan_id', 'member','loan_amnt','funded_amnt','term','int_rate','installment','issue_d','loan_status','purpose','title')
loans.repartition(1).write.format('csv')\
                            .option('header','true')\
                            .mode('overwrite')\
                            .option('path','/user/anil/lendingClubProject/loans_data')\
                            .save()

In [21]:
#creating dataset for loans_repayments 
loansRepayments  = loansRawData.withColumnRenamed('id','loan_id').select('loan_id','total_rec_prncp','total_rec_int','total_rec_late_fee','total_pymnt','last_pymnt_amnt','last_pymnt_d','next_pymnt_d')
loansRepayments.repartition(1).write.format('csv')\
                            .option('header','true')\
                            .mode('overwrite')\
                            .option('path','/user/anil/lendingClubProject/loans_repayments')\
                            .save()

In [22]:
#creating dataset for loans_defaulters
loansDefaulters = loansRawData.withColumnRenamed('emp_id','member').select('member','delinq_2yrs','delinq_amnt','pub_rec','pub_rec_bankruptcies','inq_last_6mths','total_rec_late_fee','mths_since_last_delinq','mths_since_last_record')
loansDefaulters.repartition(1).write.format('csv')\
                            .option('header','true')\
                            .mode('overwrite')\
                            .option('path','/user/anil/lendingClubProject/loans_defaulters')\
                            .save()